Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/11/05 22:30:17 WARN Utils: Your hostname, ahmed-refat-VirtualBox, resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
25/11/05 22:30:17 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.3.jar!/org/apache/ivy/core/settings/ivysettings.xml
Ivy Default Cache set to: /home/ahmed-refat/.ivy2.5.2/cache
The jars for the packages stored in: /home/ahmed-refat/.ivy2.5.2/jars
org.apache.spark#spark-sql-kafka-0-10_2.13 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-2b3e4980-99d5-4ccb-896e-03903d271ece;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.13;4.0.1 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.13;4.0.1 in central
	found org.apache.kafka#kafka-clients;3.9.1 in central
	found org.lz4#lz4-java;1.8.0 in centra

✅ Spark initialized successfully!
Spark version: 4.0.1


In [2]:
from kafka import KafkaProducer
import time

# إنشاء producer متصل بالكافكا على localhost
producer = KafkaProducer(bootstrap_servers='localhost:9092')

# إرسال 5 رسائل بسيطة لتوبيك اسمه 'simple-topic'
for i in range(5):
    message = f"Hello from Kafka {i}".encode('utf-8')
    producer.send('simple-topic', message)
    print(f"Sent: {message}")
    time.sleep(1)

producer.flush()
producer.close()


Sent: b'Hello from Kafka 0'
Sent: b'Hello from Kafka 1'
Sent: b'Hello from Kafka 2'
Sent: b'Hello from Kafka 3'
Sent: b'Hello from Kafka 4'


In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# إنشاء SparkSession
spark = SparkSession.builder \
    .appName("SimpleKafkaRead") \
    .getOrCreate()

# قراءة stream من Kafka
df = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("subscribe", "simple-topic") \
    .option("startingOffsets", "earliest") \
    .load()

# تحويل value من bytes إلى string
messages = df.selectExpr("CAST(value AS STRING) as message")

# عرض النتائج في الـ console
query = messages.writeStream \
    .outputMode("append") \
    .format("console") \
    .start()

query.awaitTermination()


25/11/05 22:31:40 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.
25/11/05 22:31:44 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-cfb6296a-00b9-44b7-b2fa-6175235d4fbd. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
25/11/05 22:31:45 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+------------------+
|           message|
+------------------+
|Hello from Kafka 0|
|Hello from Kafka 1|
|Hello from Kafka 2|
|Hello from Kafka 3|
|Hello from Kafka 4|
|Hello from Kafka 0|
|Hello from Kafka 1|
|Hello from Kafka 2|
|Hello from Kafka 3|
|Hello from Kafka 4|
|Hello from Kafka 0|
|Hello from Kafka 1|
|Hello from Kafka 2|
|Hello from Kafka 3|
|Hello from Kafka 4|
|Hello from Kafka 0|
|Hello from Kafka 1|
|Hello from Kafka 2|
|Hello from Kafka 3|
|Hello from Kafka 4|
+------------------+
only showing top 20 rows


ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/home/ahmed-refat/miniconda3/envs/pyspark_env/lib/python3.10/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/home/ahmed-refat/miniconda3/envs/pyspark_env/lib/python3.10/site-packages/py4j/clientserver.py", line 535, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/home/ahmed-refat/miniconda3/envs/pyspark_env/lib/python3.10/socket.py", line 717, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt
ERROR:py4j.clientserver:Exception occurred while shutting down connection
Traceback (most recent call last):
  File "/home/ahmed-refat/miniconda3/envs/pyspark_env/lib/python3.10/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/home/ahmed-refat/miniconda3/envs/pyspark_env/lib/python3.10/site-packages/py4j/clientserve

KeyboardInterrupt: 